In [1]:
import os
import pandas as pd
from shutil import copy2

def convert_to_yolo(row, width, height):
    x1 = row['Roi.X1']
    y1 = row['Roi.Y1']
    x2 = row['Roi.X2']
    y2 = row['Roi.Y2']

    box_width = x2 - x1
    box_height = y2 - y1
    center_x = x1 + box_width / 2
    center_y = y1 + box_height / 2

    center_x /= width
    center_y /= height
    box_width /= width
    box_height /= height

    return center_x, center_y, box_width, box_height

def process_csv(csv_path, images_root_dir, output_images_dir, output_labels_dir):
    data_csv = pd.read_csv(csv_path)
    for index, row in data_csv.iterrows():
        image_path = row['Path']
        image_name = os.path.basename(image_path)
        txt_name = image_name.replace('.png', '.txt')

        width, height = row['Width'], row['Height']
        center_x, center_y, box_width, box_height = convert_to_yolo(row, width, height)
        cls = row['ClassId']

        src = os.path.join(images_root_dir, image_path)
        dst = os.path.join(output_images_dir, image_name)

        os.makedirs(output_images_dir, exist_ok=True)

        if os.path.exists(src):
            copy2(src, dst)
        else:
            continue

        label_path = os.path.join(output_labels_dir, txt_name)
        os.makedirs(output_labels_dir, exist_ok=True)
        with open(label_path, 'w') as f:
            f.write(f'{cls} {center_x} {center_y} {box_width} {box_height}\n')

train_csv_path = '../input/gtsrb-german-traffic-sign/Train.csv'
test_csv_path = '../input/gtsrb-german-traffic-sign/Test.csv'
images_root_dir = '../input/gtsrb-german-traffic-sign'  
output_train_images_dir = 'datasets/train/images'
output_train_labels_dir = 'datasets/train/labels'
output_test_images_dir = 'datasets/val/images'
output_test_labels_dir = 'datasets/val/labels'

process_csv(train_csv_path, images_root_dir, output_train_images_dir, output_train_labels_dir)
process_csv(test_csv_path, images_root_dir, output_test_images_dir, output_test_labels_dir)


In [2]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5689.3/8062.4 GB disk)


In [3]:
!nvidia-smi

Mon Jun 10 13:21:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# !yolo detect mode=train data=/kaggle/input/data-yaml/data.yaml model=yolov8s.yaml epochs=20 imgsz=512 batch=64

In [5]:
!yolo detect train data=/kaggle/input/data-yaml/data.yaml model=yolov8n.pt epochs=20 imgsz=800 batch=64


100%|██████████████████████████████████████| 6.23M/6.23M [00:00<00:00, 73.4MB/s]
New https://pypi.org/project/ultralytics/8.2.30 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/input/data-yaml/data.yaml, epochs=20, patience=50, batch=64, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_s